<a href="https://colab.research.google.com/github/AdhithJCB/bdmc-demo/blob/main/notebooks/01_biomolecular_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install biopython py3Dmol

In [ ]:
import urllib.request

url = "https://raw.githubusercontent.com/AdhithJCB/bdmc-demo/main/data/1ubq.pdb"
out = "1ubq.pdb"
urllib.request.urlretrieve(url, out)

print("Downloaded:", out)


In [ ]:
from Bio.PDB import PDBParser

parser = PDBParser(QUIET=True)
structure = parser.get_structure("ubq", "1ubq.pdb")

model = next(structure.get_models())
chains = list(model.get_chains())

print("Chains:", [c.id for c in chains])
for c in chains:
    residues = [r for r in c.get_residues() if r.id[0] == " "]
    print(f"Chain {c.id}: {len(residues)} residues")

In [ ]:
import numpy as np

chain = chains[0]
residues = [r for r in chain.get_residues() if r.id[0] == " " and "CA" in r]

coords = np.array([r["CA"].get_coord() for r in residues])
res_ids = [r.id[1] for r in residues]  #residue numbers

#Pick a residue index to analyze (interactive portion)
i = 10  #change here!
ref = coords[i]
dists = np.linalg.norm(coords - ref, axis=1)

#show closest residues
order = np.argsort(dists)
closest = [(res_ids[j], float(dists[j])) for j in order[1:11]]
closest


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(dists, bins=20)
plt.title(f"Distances from residue index {i} (CA atoms)")
plt.xlabel("Distance (Å)")
plt.ylabel("Count")
plt.show()

In [ ]:
import py3Dmol

view = py3Dmol.view(query='pdb:1ubq')
view.setStyle({'cartoon': {'color':'spectrum'}})

#highlight the selected residue in red
sel_resnum = res_ids[i]
view.setStyle({'resi': str(sel_resnum)}, {'stick': {'color': 'red'}})

view.zoomTo()
view.show()
